# Работа с BERT

Загружаем модель


Используется код из следующих источников:
1. https://github.com/google-research/bert
2. https://github.com/google-research/bert/blob/master/tokenization.py
3. https://habr.com/ru/post/436878/

In [68]:
# Запустим файл от создателей BERT
%run tokenization.py

In [40]:
import sys
import codecs
import numpy as np
from keras_bert import load_trained_model_from_checkpoint

# папка, куда распаковали преодобученную нейросеть BERT
folder = '/Users/annaaksenova/Desktop/multi_cased_L-12_H-768_A-12'

config_path = "/Users/annaaksenova/Desktop/multi_cased_L-12_H-768_A-12"+'/bert_config.json'
checkpoint_path = "/Users/annaaksenova/Desktop/multi_cased_L-12_H-768_A-12"+'/bert_model.ckpt'
vocab_path = "/Users/annaaksenova/Desktop/multi_cased_L-12_H-768_A-12"+'/vocab.txt'

Using TensorFlow backend.


In [69]:
tokenizer = FullTokenizer(vocab_file=vocab_path, do_lower_case=False)

In [70]:
model = load_trained_model_from_checkpoint(config_path, checkpoint_path, training=True)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 512)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 512, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 512, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [71]:
sentence = "Пересказывая содержание книги, Лена поведала о стране, где живут феи, эльфы и гоблины, [MASK] [MASK] выдуманные сказочные существа."

print(sentence)

Пересказывая содержание книги, Лена поведала о стране, где живут феи, эльфы и гоблины, [MASK] [MASK] выдуманные сказочные существа.


In [72]:
sentence = sentence.replace(' [MASK] ','[MASK]'); sentence = sentence.replace('[MASK] ','[MASK]'); sentence = sentence.replace(' [MASK]','[MASK]')  # удаляем лишние пробелы
sentence = sentence.split('[MASK]')             # разбиваем строку по маске
tokens = ['[CLS]']                              # фраза всегда должна начинаться на [CLS]
# обычные строки преобразуем в токены с помощью tokenizer.tokenize(), вставляя между ними [MASK]
for i in range(len(sentence)):
    if i == 0:
        tokens = tokens + tokenizer.tokenize(sentence[i]) 
    else:
        tokens = tokens + ['[MASK]'] + tokenizer.tokenize(sentence[i]) 
tokens = tokens + ['[SEP]']                     # фраза всегда должна заканчиваться на [SEP] 

In [73]:
token_input = tokenizer.convert_tokens_to_ids(tokens)  

In [74]:
token_input = token_input + [0] * (512 - len(token_input))

In [75]:
mask_input = [0]*512
for i in range(len(mask_input)):
    if token_input[i] == 103:
        mask_input[i] = 1

In [76]:
seg_input = [0]*512

In [77]:
token_input = np.asarray([token_input])
mask_input = np.asarray([mask_input])
seg_input = np.asarray([seg_input])

In [78]:
predicts = model.predict([token_input, seg_input, mask_input])[0] 
predicts = np.argmax(predicts, axis=-1)
predicts = predicts[0][:len(tokens)]    # отрезаем начало фразы, длиной как исходная фраза, чтобы отсечь случайные выбросы среди нулей дальше

In [79]:
out = []
# добавляем в out только слова в позиции [MASK], которые маскированы цифрой 1 в mask_input
for i in range(len(mask_input[0])):
    if mask_input[0][i] == 1:                       # [0][i], т.к. сеть возвращает batch с формой (1,512), где в первом элементе наш результат
        out.append(predicts[i]) 

out = tokenizer.convert_ids_to_tokens(out)          # индексы в текстовые токены
out = ' '.join(out)                                 # объединяем токены в строку с пробелами
out = printable_text(out)              # в удобочитаемый текст
out = out.replace(' ##','')                         # объединяем разъединенные слова: "при ##шел" -> "пришел"

In [80]:
print('Result:', out)

Result: где также


# Посмотрим, как BERT работает с примерами из нашей обучающей выборки

In [81]:
import openpyxl

In [82]:
## Работа с таблицей по векам

wb = openpyxl.load_workbook('/Users/annaaksenova/Desktop/Работа к Кувшинской/То есть выборка по векам.xlsx')
sheet = wb['21'] # Имя лист

### Количество угаданных масок *то*

In [83]:
v1 = []
# Собираем примеры из таблицы
for v in sheet.iter_rows(min_row=2, min_col=21, max_col=21, max_row=101, values_only=True):
    if v[0] == None:
        v1.append('')
    else:
        v1.append(v[0].replace(u'\xa0', ' ').strip())
v2 = []
for v in sheet.iter_rows(min_row=2, min_col=22, max_col=22, max_row=1001, values_only=True):
    if v[0] == None:
        v2.append('')
    else:
        item = v[0].replace(u'\xa0', ' ')
        item = item.strip()
        item = item.replace('  ', '')
        item = re.sub('\[.+\]', '', item)
        # Заменяем маской ТО
        item = re.sub(r'\b([Тт]о есть', r'[MASK] есть', item)  # \b убирает случаи что есть
        v2.append(item)
values = [v1[i]+v2[i] for i in range(100)]

In [84]:
def bert_parse(sentence):
    sentence = sentence.replace(' [MASK] ','[MASK]'); sentence = sentence.replace('[MASK] ','[MASK]'); sentence = sentence.replace(' [MASK]','[MASK]')  # удаляем лишние пробелы
    sentence = sentence.split('[MASK]')             # разбиваем строку по маске
    tokens = ['[CLS]']                              # фраза всегда должна начинаться на [CLS]
# обычные строки преобразуем в токены с помощью tokenizer.tokenize(), вставляя между ними [MASK]
    for i in range(len(sentence)):
        if i == 0:
            tokens = tokens + tokenizer.tokenize(sentence[i]) 
        else:
            tokens = tokens + ['[MASK]'] + tokenizer.tokenize(sentence[i]) 
    tokens = tokens + ['[SEP]']      
    token_input = tokenizer.convert_tokens_to_ids(tokens) 
    token_input = token_input + [0] * (512 - len(token_input))
    mask_input = [0]*512
    for i in range(len(mask_input)):
        if token_input[i] == 103:
            mask_input[i] = 1
    seg_input = [0]*512
    token_input = np.asarray([token_input])
    mask_input = np.asarray([mask_input])
    seg_input = np.asarray([seg_input])
    predicts = model.predict([token_input, seg_input, mask_input])[0] 
    predicts = np.argmax(predicts, axis=-1)
    predicts = predicts[0][:len(tokens)]
    out = []
    # добавляем в out только слова в позиции [MASK], которые маскированы цифрой 1 в mask_input
    for i in range(len(mask_input[0])):
        if mask_input[0][i] == 1:                       # [0][i], т.к. сеть возвращает batch с формой (1,512), где в первом элементе наш результат
            out.append(predicts[i]) 

    out = tokenizer.convert_ids_to_tokens(out)          # индексы в текстовые токены
    out = ' '.join(out)                                 # объединяем токены в строку с пробелами
    out = printable_text(out)              # в удобочитаемый текст
    out = out.replace(' ##','')
    return out

In [85]:
predictions = []
for i in range(100):
    a = bert_parse(values[i])
    predictions.append(a)

In [87]:
len([i for i in predictions if i.lower()=='то'])

80

80 из 100 примеров угаданы верно

### Количество верно угаданных масок *есть*

In [102]:
v1 = []
# Собираем примеры из таблицы
for v in sheet.iter_rows(min_row=2, min_col=21, max_col=21, max_row=101, values_only=True):
    if v[0] == None:
        v1.append('')
    else:
        v1.append(v[0].replace(u'\xa0', ' ').strip())
v2 = []
for v in sheet.iter_rows(min_row=2, min_col=22, max_col=22, max_row=1001, values_only=True):
    if v[0] == None:
        v2.append('')
    else:
        item = v[0].replace(u'\xa0', ' ')
        item = item.strip()
        item = item.replace('  ', '')
        item = re.sub('\[.+\]', '', item)
        # Заменяем маской ТО
        item = re.sub(r'\b([Тт])о есть', r'\1о [MASK]', item)  # \b убирает случаи что есть
        v2.append(item)
values = [v1[i]+v2[i] for i in range(100)]

In [103]:
values[:2]

['Я засунула пластинку и блюдо в один пакет, положила под сиденье и ― надо знать наши самолёты советских времён, лишённые кондиционеров, ― в конце пути в Ереван достала из-под сиденья пластинку, принявшую форму таза, то [MASK] блюда.',
 'Само название места происходит от того, что здесь врачевали «косимых» ― то [MASK] увечных.']

In [104]:
predictions = []
for i in range(100):
    a = bert_parse(values[i])
    predictions.append(a)

In [106]:
len([i for i in predictions if i.lower()=='есть'])

72

72 из 100 примеров угаданы верно